In [17]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tushare as ts
import os 
import time
import datetime as dt
import sqlite3
from sqlalchemy import create_engine

In [18]:
def f(row):
   return row['high']-row['low']

In [19]:
##如果文件不存在，会自动在当前目录创建:
print(os.getcwd())
dbPath=os.getcwd()+'\data\stock.db'
print(dbPath)
conn = sqlite3.connect(dbPath)
engine = create_engine('sqlite:///'+dbPath, echo = False) #注5
stockList=ts.get_stock_basics()
print(stockList)
stockList.to_sql('stocks',engine,if_exists='replace', index=True)

D:\Coding\stock
D:\Coding\stock\data\stock.db
          name industry area       pe  outstanding  totals  totalAssets  \
code                                                                      
002177    御银股份     电脑设备   广东   236.36         6.03    7.61    202337.41   
600405     动力源     电气设备   北京     0.00         5.53    5.62    297222.53   
300420    五洋停车     机械基件   江苏    48.37         2.50    5.11    154036.91   
002226    江南化工     化工原料   安徽    48.06         9.11    9.11    514374.44   
002280    联络互动     软件服务   浙江   364.52        18.50   21.77   1398823.00   
002194    武汉凡谷     通信设备   湖北     0.00         5.30    5.65    225832.41   
600721     百花村     生物制药   新疆    41.04         3.03    4.00    275831.75   
300369    绿盟科技     软件服务   北京     0.00         5.47    8.04    331088.44   
300344    太空板业     其他建材   北京     0.00         1.93    3.38    141963.77   
603712     七一二     通信设备   天津   116.47         1.00    7.72    348015.50   
300378    鼎捷软件     软件服务   上海    58.66         2.61    

In [20]:
code='600660'
data=ts.get_k_data(code=code,start='2018-01-01',end='2018-03-09')
#设置date列为索引
data.set_index('date',inplace=True)
#增加字段,计算列
#如何使用应用函数多列
##https://codeday.me/bug/20170717/38276.html
data["change"]=data.apply(f, axis=1)
print(data)

## 计算平均
print(data["change"].mean())

## 获取当日所有股票的数据
## dt.now().weekday是返回今天是星期几，5代表星期六，6代表星期天
current_date = time.strftime("%Y-%m-%d",time.localtime())
if dt.datetime.now().weekday() == 5: #注1
    today = str(pd.Timestamp(dt.datetime.now())-pd.Timedelta(days = 1))[:10]  #注2
elif dt.datetime.now().weekday() == 6:
    today = str(pd.Timestamp(dt.datetime.now())-pd.Timedelta(days = 2))[:10] 
else:
    today = str(pd.Timestamp(dt.datetime.now()))[:10]
## 中国股票市场的今天
print(today)



print(current_date)
allPath=os.getcwd()+'\data\get_today_all.csv'
print(allPath)

#if os.path.exists(allPath):
    #toady_all=pd.read_csv('allPath',encoding='utf-8')
#else:
    #toady_all=ts.get_today_all()
    #toady_all.to_csv(allPath)
    

#获取破指定天数内的新高 比如破60日新高  
info=ts.get_stock_basics()  
  
def loop_all_stocks():  
    for EachStockID in info.index:  
         if is_break_high(EachStockID,60):  
             print("High price on")
             print(EachStockID)
             print(info.ix[EachStockID]['name'])  
  
  
  
def is_break_high(stockID,days):  
    end_day=dt.date(dt.datetime.today().year,dt.datetime.today().month,dt.datetime.today().day)  
    days=days*7/5  
    #考虑到周六日非交易  
    start_day=end_day-dt.timedelta(days)  
  
    start_day=start_day.strftime("%Y-%m-%d")  
    end_day=end_day.strftime("%Y-%m-%d")  
    df=ts.get_h_data(stockID,start=start_day,end=end_day)  
  
    period_high=df['high'].max()  
    #print period_high  
    today_high=df.iloc[0]['high']  
    #这里不能直接用 .values  
    #如果用的df【：1】 就需要用.values  
    #print today_high  
    if today_high>=period_high:  
        return True  
    else:  
        return False  
  
#loop_all_stocks()  


             open  close   high    low    volume    code  change
date                                                            
2018-01-02  29.10  29.89  29.90  29.10  153337.0  600660    0.80
2018-01-03  29.90  29.54  30.00  29.37  129270.0  600660    0.63
2018-01-04  29.53  29.62  29.77  29.27  102790.0  600660    0.50
2018-01-05  29.59  29.36  29.60  29.11  136923.0  600660    0.49
2018-01-08  29.31  29.79  29.98  28.81  188204.0  600660    1.17
2018-01-09  29.80  29.64  29.88  29.50   98384.0  600660    0.38
2018-01-10  29.58  30.00  30.00  29.40  107345.0  600660    0.60
2018-01-11  30.00  29.79  30.22  29.60   82718.0  600660    0.62
2018-01-12  29.86  29.39  29.95  29.20   99721.0  600660    0.75
2018-01-15  29.38  28.49  29.38  28.45  170221.0  600660    0.93
2018-01-16  28.42  28.88  28.95  28.18  129035.0  600660    0.77
2018-01-17  28.75  28.15  28.75  27.80  194122.0  600660    0.95
2018-01-18  28.18  28.29  28.35  28.05  102734.0  600660    0.30
2018-01-19  28.28  28.41 